## 7-1. 人の移動を Simulation してみよう
空間上に個体を Random に配置して個体の動きを加える。
それぞれの個体について「時間発展方程式」によって動きを表現する際、その「変量」に対して「拘束条件」が加わる。

In [2]:
# 人の動きの Simulation を行なう##
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# Parameter設定
dt = 1.0  # １コマあたりの時間
dl = 1.0  # １コマあたりの各個体の移動距離
num_time = 100  # Simulation を行なうコマ数
num_person = 10  # Simulation を行なう人の数
x_size = 8.0  # 領域の x方向の Size
y_size = 6.0  # 領域の y方向の Size
## ※横方向に 8.0m, 縦方向に 6.0m の家具のない部屋に10人の人がいるような Image.

# 初期化（初期値設定）
list_plot = []  # Animation の各コマを格納する配列
# 各個体の初期値(x, y)を設定
x = np.zeros((num_time, num_person))
y = np.zeros((num_time, num_person))
for i in range(num_person):
    x[0, i] = np.random.rand() * x_size
    y[0, i] = np.random.rand() * y_size

# 時間発展方程式
fig = plt.figure()
for t in range(1, num_time):
    # 変量の計算（各個体がどの方向にどれだけ移動するのか）
    dx = (np.random.rand(num_person) - 0.5) * dl
    dy = (np.random.rand(num_person) - 0.5) * dl
    # 拘束条件の設定（変量が領域内に収まっているかどうかなどを評価し、誤った移動をしないための判断を行なう）
    for i in range(num_person):
        # 計算した変量(dx, dy)に従った行動を行なった際に、領域（x: 0~x_size まで留まっているか）を確認
        if ((x[t - 1, i] + dx[i] * dt) > 0) and ((x[t - 1, i] + dx[i] * dt) < x_size):
            x[t, i] = x[t - 1, i] + dx[i] * dt
        else:
            x[t, 1] = x[t - 1, i]  # もし、計算した変量従った移動を行なった際に、建物の領域からはみだしてしまうようであれば、移動は行なわない
        # 計算した変量(dx, dy)に従った行動を行なった際に、領域（y: 0~y_size まで留まっているか）を確認
        if ((y[t - 1, i] + dy[i] * dt) > 0) and ((y[t - 1, i] + dy[i] * dt) < y_size):
            y[t, i] = y[t - 1, i] + dy[i] * dt
        else:
            y[t, i] = y[t - 1, i]  # もし、計算した変量従った移動を行なった際に、建物の領域からはみだしてしまうようであれば、移動は行なわない
    # 時刻ごとの Graph描画
    img = plt.scatter(x[t], y[t], color="black")
    plt.xlim([0, x_size])
    plt.ylim([0, y_size])
    list_plot.append([img])

# Graph(Animation)描画
plt.grid()
anim = animation.ArtistAnimation(fig, list_plot, interval=200, repeat_delay=1000)
rc("animation", html="jshtml")
plt.close()
anim